In [2]:
import numpy as np
import pandas as pd 
from pandas import DataFrame
import random
from math import ceil
from math import log

Сделаем код воспроизводимым, также, в дальнейшем, во все алгоритмы будем добавлять $seed$ как $random\_state$

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)

Загрузим данные любым способом

Загрузка из гугл-диска

In [9]:
from google.colab import drive
drive.mount('/content/drive')
path_to_files = '/content/drive/My Drive/data/'

Mounted at /content/drive


Загрузка из локального хранилища

In [ ]:
path_to_files = ''

Общая загрузка

In [10]:
movie = pd.read_csv(path_to_files + 'movie.csv')
rating = pd.read_csv(path_to_files + 'rating.csv')
link = pd.read_csv(path_to_files + 'link.csv')
tag = pd.read_csv(path_to_files + 'tag.csv')
gen_tags = pd.read_csv(path_to_files + 'genome_tags.csv')
gen_scores = pd.read_csv(path_to_files + 'genome_scores.csv')


Изменим шкалу рейтинга, т.к в датасете она (min=0.5; max=5; step=0.5), сделав её (min=1; max=10; step=1)

Также, оставим только столбцы для нашей матрицы кросс-табуляции

In [ ]:
rating['rating'].unique()

array([3.5, 4. , 3. , 4.5, 5. , 2. , 1. , 2.5, 0.5, 1.5])

In [11]:
rating['rating'] = (rating['rating'] * 2).astype(int)
rating['rating'].value_counts()
rating = rating[rating.columns[:3]]

Сделаем разбиение:

Нам необходимо, чтобы в обучающей выборке был хотя бы один пример для каждого пользователя и каждого фильма, иначе наша модель будет невалидна, для заданной пары пользователь/фильм она не сможет выдать корректный ответ, ибо не будет иметь данных о ней. 

Например, в таком случае модель из $surprise$ выдаст ответ, в котором будет записано, что выдать правильный ответ было невозможно, $was \space impossible = True$


Функция, которая помещает в $train$ датасет как минимум по 1 экземпляру уникальных $userId$, по 1 экземпляру уникальных $movieId$, далее заполняет случайным образом $train$ до нужного отношения $\frac{train}{full\_dataset}$ равное $ratio$, оставшиеся данные добавляет в $valid$ и $test$ пополам

In [12]:
def train_valid_test_split(dataset : DataFrame, ratio : float) -> tuple:

    users = dataset.loc[dataset['userId'].drop_duplicates().index]
    movies = dataset.loc[dataset['movieId'].drop_duplicates().index]
    train = users.merge(movies, how="outer")
    test = dataset.drop(train.index) 

    num_to_choice = int(len(dataset) * ratio) - len(train)
    index = np.random.choice(test.index, size=num_to_choice)
    train = train.append(test.loc[index])
    test = test.drop(index)

    index = np.random.choice(test.index, size=int(len(test)/2))
    valid = test.loc[index]
    test = test.drop(index)

    return train, valid, test

Разделим датасет на $train$, $valid$ и $test$. Подбор гиперпараметров будем делать на валидации, на тесте будем фиксировать итоговое качество моделей, по которым и будем их сравнивать

In [13]:
train_set, valid_set, test_set = train_valid_test_split(rating, 0.9)

Для простого коллаборативного подхода воспользуемся $surprise$ — https://surprise.readthedocs.io/en/stable/index.html

In [ ]:
!pip install surprise

In [ ]:
from surprise import Reader
from surprise import Dataset
from surprise import AlgoBase
from surprise import Reader
from surprise import Prediction
from surprise import NMF
from surprise import accuracy

Выберем метрику: 

Возьмём как pointwise метрику - $RMSE$, чтобы можно было оценить, насколько в среднем наш алгоритм отклоняется от правильной оценки 

В качества listwise метрики возьмём $NDCG$ (https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG), чтобы можно было оценить, насколько хорошо наш алгоритм ранжирует фильмы по оценкам для пользователя в процентах. За счёт логарифмов, каждый следующий ранг начинает вносить всё меньше и меньше значимости, т.к зачастую нам важно именно выдать top-k рекомендаций, и уже не так важно, что будет дальше

В $surprise$ есть свой $RMSE$, а вот $NDCG$ необходимо будет написать самостоятельно

In [ ]:
def ndcg(predictions : list, verbose = False) -> float:

    line = [[predictions[i].est, predictions[i].r_ui] for i in range(len(predictions))]
    sl = sorted(line, key = lambda el : el[1], reverse = True)

    idcg_res = np.array([(2 ** sl[i - 1][1] - 1) / log(i + 1, 2) for i in 
                         range(1, len(sl) + 1)]).sum()

    sl = sorted(line, key = lambda el : el[0], reverse = True)
    ndcg_res = np.array([(2 ** sl[i - 1][1] - 1) / log(i + 1, 2) for i in 
                         range(1, len(sl) + 1)]).sum()

    metric_res = ndcg_res/idcg_res
    if verbose:
        print("NDCG: ", metric_res)

    return metric_res

Обучение


Округляем ответы к ближайшему целому

In [ ]:
def predict(model : AlgoBase,
            data: DataFrame) -> list:

    pred = model.test(data.to_numpy())
    return [Prediction(p.uid, p.iid, p.r_ui, round(p.est), p.details) for p in pred]

In [ ]:
def train(model : AlgoBase, 
          train_dataset : DataFrame, 
          valid_dataset : DataFrame, 
          reader : Reader,
          metrics : dict,
          verbose = False) -> dict:


    if verbose: 
        print("Train")

    model.fit(Dataset.load_from_df(train_dataset, reader).build_full_trainset())

    if verbose: 
        print("Valid")

    predicted = predict(model, valid_dataset)

    if verbose:
        print("Metrics:")

    metrics_value = {k : v(predicted, verbose=verbose) for k, v in metrics.items()}

    if verbose:
        print(metrics_value)

    return metrics_value

В качесте модели возьмём NMF

In [ ]:
reader = Reader(rating_scale=(1, 10))
metrics = {"RMSE" : accuracy.rmse, "NDCG" : ndcg}

In [ ]:
model_nmf = NMF(n_factors = 30, random_state=seed)

In [ ]:
metrics_value = train(model_nmf, train_set, valid_set, reader, metrics, verbose = True)

Train
Valid
Metrics:
RMSE: 1.7854
NDCG:  0.9592456471378288
{'RMSE': 1.7854111921626319, 'NDCG': 0.9592456471378288}


Попробуем улучшить результаты, сделав кросс-валидацию по некоторым важным параметрам, которые влияют на сходимость и качество модели

Отбирать лучшую модель будем по $NDCG$

In [ ]:
param_grid = {'n_factors' : [15, 30, 50],
              'reg_pu' : [0.04, 0.06, 0.08],
              'reg_qi' : [0.04, 0.06, 0.08]
              }

best_ndcg = 0
best_params = {'n_factors' : 0, 'reg_pu' : 0, 'reg_qi' : 0}

for reg_pu in param_grid['reg_pu']:

    for reg_qi in param_grid['reg_qi']: 

        for n_factor in param_grid['n_factors']:

            model = NMF(n_factors=n_factor, random_state=seed,
                        reg_qi=reg_qi, reg_pu=reg_pu)
            
            metrics_value = train(model, train_set, valid_set, reader, metrics, verbose=True)
            
            print("num factors = ", n_factor)
            print("users regularization = ", reg_pu) 
            print("items regularization = ", reg_qi)

            print("Metrics:")
            print(metrics_value)
            print("\n")
            if (metrics_value['NDCG'] > best_ndcg):

                best_ndcg = metrics_value['NDCG']
                best_params['n_factors'] = n_factor
                best_params['reg_pu'] = reg_pu
                best_params['reg_qi'] = reg_qi

In [ ]:
print("Best params: ")
print(best_params)

In [ ]:
model = NMF(n_factors=30, random_state=seed,
                        reg_qi=0.06, reg_pu=0.06)
            
train(model, train_set, valid_set, reader, metrics)

{'NDCG': 0.9592456471378288, 'RMSE': 1.7854111921626319}

Протестируем модель

In [ ]:
pred = predict(model, test_set)   
ndcg(pred, True)

NDCG:  0.9598020553583039


0.9598020553583039

$NDCG = 0.9598$

Попробуем не только коллаборативный, но также и контекстный подход

Эту задачу будем решать с помощью $LightFM$ — https://making.lyst.com/lightfm/docs/home.html



In [14]:
!pip install lightfm

     |████████████████████████████████| 317kB 5.1MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705341 sha256=6b2e96d3fa21e2abd9a2088213264c6771da6e55d4da66dd61b64026c2c98c11
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [15]:
from lightfm import LightFM
from scipy.sparse.coo import coo_matrix

In [16]:
def cross_tabulation_to_sparse(dataset : DataFrame) -> coo_matrix:

    row = dataset['userId'].to_numpy(dtype='int')
    col = dataset['movieId'].to_numpy(dtype='int')
    data = dataset['rating'].to_numpy(dtype='int')

    return coo_matrix((data, (row, col)))

Для контекстного подхода, кроме матрицы кросс-табуляции нам также требуются и какие-то признаки об объектах и пользователях

Из информации о фильмах будем брать год фильма, а также One-hot-encoding представление жанров

In [ ]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Будем собирать информацию о фильмах

In [17]:
movie_info = rating.merge(movie, how="left", on="movieId")[['movieId', 'title', 'genres']]

Добавм информацию о годе выпуска фильма (0 - если не указано)

In [18]:
import re
movie_info['Year'] = movie_info['title'].apply(lambda s: (['0'] + re.findall("(\d{4})", s))[-1]).astype('int')

Получим множество всех жанров

In [19]:
genres = set()
for l_g in movie_info['genres'].unique():

    for genre in l_g.split('|'):
        genres.add(genre)

Сделаем one-hot-encoding кодирование жанров фильмов, добавив каждому фильму 1 в соответствующий столбец с жанром, если этот фильм принадлежит данному жанру, иначе 0

Полученные матрицы будут огромны, порядка $16 * 10^6 * 21$ из-за one-hot-encoding'а, поэтому все действия нужно делать очень аккуратно, лишнее копирование и получим переполнение ОЗУ. И конечно же, все операции будут выполняться с разреженными матрицами

Разделим информацию о фильмах

In [20]:
train_movie_info = movie_info.loc[train_set.index]
valid_movie_info = movie_info.loc[valid_set.index]
test_movie_info = movie_info.loc[test_set.index]

Удалим индексы фильмов, т.к матрица по одной из размерностей будет совпадать с матрицей кросс-табуляции, в которой содержится информация об этом

In [21]:
train_movie_info.drop(columns="movieId", inplace=True)
valid_movie_info.drop(columns="movieId", inplace=True)
test_movie_info.drop(columns="movieId", inplace=True)

In [22]:
train_ohe_movie = pd.DataFrame()
valid_ohe_movie = pd.DataFrame()
test_ohe_movie = pd.DataFrame()

In [23]:
train_ohe_movie['Year'] = train_movie_info['Year']
valid_ohe_movie['Year'] = valid_movie_info['Year']
test_ohe_movie['Year'] = test_movie_info['Year']

Делаем one-hot-encoding

In [24]:
for genre in genres:

    train_ohe_movie[genre] = train_movie_info['genres'].apply(lambda s: int(bool(s.find(genre) + 1)))
    valid_ohe_movie[genre] = valid_movie_info['genres'].apply(lambda s: int(bool(s.find(genre) + 1)))
    test_ohe_movie[genre] = test_movie_info['genres'].apply(lambda s: int(bool(s.find(genre) + 1)))
    print(genre, " was added in data")

Children  was added in data
Documentary  was added in data
Comedy  was added in data
Film-Noir  was added in data
Sci-Fi  was added in data
Drama  was added in data
Mystery  was added in data
(no genres listed)  was added in data
Romance  was added in data
IMAX  was added in data
Musical  was added in data
Animation  was added in data
Fantasy  was added in data
Western  was added in data
Action  was added in data
Crime  was added in data
Horror  was added in data
Thriller  was added in data
War  was added in data
Adventure  was added in data


Можно брать оценки и популярность фильма из omdbapi, или tmdb, но они оба ставят ограничения на free api, поэтому разметить наш датасет (20M) не получится, но в теории возможно

In [ ]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Данная информация содержит слишком разные теги, хотя вообще, можно получать какой-то эмбеддинг из текстового тега и добавлять к нашей информации

In [ ]:
tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


Сделаем все матрицы разреженными

In [25]:
train_movie_info = (train_ohe_movie.astype(pd.SparseDtype(int, fill_value=0))).sparse.to_coo()
valid_movie_info = (valid_ohe_movie.astype(pd.SparseDtype(int, fill_value=0))).sparse.to_coo()
test_movie_info = (test_ohe_movie.astype(pd.SparseDtype(int, fill_value=0))).sparse.to_coo()

In [26]:
train_set_coo = cross_tabulation_to_sparse(train_set)
valid_set_coo = cross_tabulation_to_sparse(valid_set)
test_set_coo = cross_tabulation_to_sparse(test_set)

Обучение и подбор модели

Из-за огромных размеров датасета, делать кросс-валидацию в ноутбуке сложно, лучшие гиперпараметры подбирались итеративно по $NDCG$

In [27]:
model = LightFM(loss='warp', no_components = 30, learning_rate = 0.01, random_state=seed)

In [28]:
model.fit(train_set_coo,
          user_features=train_movie_info,
          epochs=80,
          verbose=True)

Epoch: 100%|██████████| 80/80 [1:25:54<00:00, 64.43s/it]


In [ ]:
predictions = model.predict(valid_set['userId'].to_numpy(),
                            valid_set['movieId'].to_numpy(),
                            user_features=valid_movie_info)

Немного поменяем метрику $NDCG$ под эту библиотеку

In [29]:
def ndcg_lightfm(predictions : np.array, y_true : np.array , verbose = False) -> float:

    line = [[predictions[i], y_true[i]] for i in range(len(predictions))]
    
    sl = sorted(line, key = lambda el : el[1], reverse = True)
    idcg_res = np.array([(2 ** sl[i - 1][1] - 1) / log(i + 1, 2) for i in 
                         range(1, len(sl) + 1)]).sum()

    sl = sorted(line, key = lambda el : el[0], reverse = True)
    ndcg_res = np.array([(2 ** sl[i - 1][1] - 1) / log(i + 1, 2) for i in 
                         range(1, len(sl) + 1)]).sum()

    metric_res = ndcg_res/idcg_res
    if verbose:
        print("NDCG: ", metric_res)

    return metric_res

Заметим, что $LightFM$ выдаёт только список рангов в другом пространстве, а не значения рейтингов, как в нашем датасете, поэтому её нельзя тестировать на $RMSE$, чтобы оценить модель интуитивно - на отклонение оценок от оптимального

In [ ]:
predictions

In [ ]:
ndcg_lightfm(predictions, valid_set['rating'].to_numpy(),verbose=True)

Протестируем нашу модель

In [30]:
predictions = model.predict(test_set['userId'].to_numpy(),
                            test_set['movieId'].to_numpy(),
                            user_features=test_movie_info)

In [31]:
ndcg_lightfm(predictions, test_set['rating'].to_numpy(),verbose=True)

NDCG:  0.9266153695432057


0.9266153695432057

$NDCG = 0.9266$

По метрике $NDCG$, можно сказать, что простая коллаборативная модель ранжирует лучше, чем та, которая кроме этого использует контекст.
Возможные причины этого: 

1.   Возможно, модель лучше покажет себя на какой-то другой метрике, которую она конкретно оптимизирует, например для $warp$ лосса, который мы использовали во второй модели, больше подходит $precision@k$
2.   Плохой подбор параметров
3.   Недостаточное количество эпох для обучения
4.   Плохой выбор пространства признаков (One-hot-encoding жанров) для данных условий, из-за чего модель сложно (большой размер данных с ограниченным ОЗУ, порядка $16 * 10^6 * 21$ значений) и долго обучать, то есть эта причина влечёт за собой вторую и третью





Проверим статистическую значимость результатов на $NMF$ модели на уровне значимости $p_{value} = 0.05$

Гипотеза $H_0$: распределения предсказаний и настоящих рейтингов не отличаются

Гипотеза $H_1$: распределения предсказаний и настоящих рейтингов отличаются

In [ ]:
train(model_nmf, train_set, valid_set, reader, metrics, True)

Train
Valid
Metrics:
RMSE: 1.7854
NDCG:  0.9592456471378288
{'RMSE': 1.7854111921626319, 'NDCG': 0.9592456471378288}


{'NDCG': 0.9592456471378288, 'RMSE': 1.7854111921626319}

In [ ]:
from scipy.stats import chisquare

def stat_test(pred : Prediction):

    f_exp_obs = np.array([[p.est, p.r_ui] for p in pred])
    return chisquare(f_exp = f_exp_obs[:, 1], f_obs = f_exp_obs[:, 0])

In [ ]:
pred = predict(model_nmf, test_set)

In [ ]:
test_value = stat_test(pred)
p_value = 0.05

In [ ]:
if test_value.pvalue > p_value:
    print("Нельзя отвергнуть гипотезу H_0")
else:
    print("Гипотеза H_0 отвергается, принимается гипотеза H_1")

Нельзя отвергнуть гипотезу H_0


Выводы


1.   От метрики очень сильно зависит отбор кандидатов моделей, нужно внимательно проанализировать, что мы хотим получать от модели в результате, и уже от этого выбирать метрику
2.   Даже простая модель может выдавать неплохие результаты
3.   Сложные модели требуют тщательной настройки гиперпараметров
4.   Сложные модели тяжело обучать, потому что у них больше пространство признаков $=>$ требуется больше времени для обучения и больше памяти для данных
5.   Из всего выше, следует, что самый оптимальный вариант для рекомендательной системы — обучать простую модель, которая будет выбирать $top-k$ лучших кандидатов, и их подавать в более тяжеловесную модель с большим количеством признаков, которая будет более точно ранжировать этот список

